# ROS Action

Tu zaczyna się akcja ;-). Przeznaczone do skomplikowanych działań zachowują się jak service (zadajemy cel i wreszcie dostajemy wynik) jak i topic (dostajemy sprzężenie zwrotne).

Większość akcji robota związanych jest z ruchem -- przemieszczaniem się robota w pomieszczeniu (Navigate) albo ruchem ramienia (Moveit). Robot otrzymuje zadanie -- np. dojedź do punktu i informuje o przebiegu wykonywania zadania.


Nav ma akcje zgrabnie opakowane za pomocą simple nav ale mamy wtedy komunikację synchroniczną


Akcja różni się od serwisu przez długość swojego trwania. Przez to, że nie wykonuje się od razu, z natury lepiej jest obsługiwać ją asynchronicznie plus chcemy mieć feedback z tego jak idzie zadanie

<img src="./images/action.jpg" width="30%">
Schemat wygląda tak:

- dajemy robocikowi zadanie i callbacki. Dostajemy informację o samym przyjęciu zadania jako callback
- w czasie wykonywania zadania korzysta z callbackow
- dostajemy informację o wykonaniu lub porażce zadania

In [1]:
import rclpy
from rclpy.node import Node

from std_msgs.msg import String


class MinimalNode(Node):
    def __init__(self):
        super().__init__("service_node")


try:
    rclpy.init()

    minimal_node = MinimalNode()
except RuntimeError:
    pass

In [14]:
# from geometry_msgs.action import
from nav2_msgs.action import NavigateToPose
from rclpy.action import ActionClient
from geometry_msgs.msg import PoseStamped

navigate_action = ActionClient(minimal_node, NavigateToPose, "/navigate_to_pose")

In [13]:
!ros2 action info navigate_to_pose

Action: navigate_to_pose
Action clients: 4
    /bt_navigator
    /waypoint_follower
    /rviz
    /_
Action servers: 1
    /bt_navigator


In [ ]:
def print_feedback_goal(future):
    try:
        feedback = future.feedback
        # print("distance {}".format(feedback.distance_remaining))
        print("feedback {}".format(future.feedback))
    except:
        pass


def history_success(future):
    print("done")
    print(
        "we have achieved the goal {}, {}".format(
            future.result(), future.result().result
        )
    )


def received_task(future):
    goal_handle = future.result()

    if not goal_handle.accepted:
        print("not accepted")
        return
    else:
        print("accepted goal")

    get_result_future = goal_handle.get_result_async()

    get_result_future.add_done_callback(history_success)


navigate_pose_goal = NavigateToPose.Goal()
goal_pose = PoseStamped()
goal_pose.header.stamp = minimal_node.get_clock().now().to_msg()
goal_pose.pose.position.x = 4.0
goal_pose.pose.position.y = 0.0
goal_pose.pose.orientation.z = 0.7
goal_pose.pose.orientation.w = 0.71
navigate_pose_goal.pose = goal_pose
goal_future_pose = navigate_action.send_goal_async(
    navigate_pose_goal, feedback_callback=print_feedback_goal
)
goal_future_pose.add_done_callback(received_task)

In [24]:
import subscription_threaded

threaded_spinner = subscription_threaded.ThreadedSpinner(minimal_node)

In [71]:
# możemy albo w osobnym wątku albo spin_until_future_complete(minimal_node, send_goal_future)


threaded_spinner.spin_in_thread()

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1408, nanosec=118000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=-0.0795978485511517, y=0.054837724023454826, z=0.010016145546093112), orientation=geometry_msgs.msg.Quaternion(x=-0.0009983647298232605, y=-0.0008581720648857926, z=0.7990599291633425, w=0.6012499450428642))), navigation_time=builtin_interfaces.msg.Duration(sec=34, nanosec=430000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=18, distance_remaining=0.0)
done
we have achieved the goal nav2_msgs.action.NavigateToPose_GetResult_Response(status=6, result=nav2_msgs.action.NavigateToPose_Result(result=std_msgs.msg.Empty())), nav2_msgs.action.NavigateToPose_Result(result=std_msgs.msg.Empty())


In [ ]:
goal_handle = goal_future_pose.result()

In [ ]:
send_goal_future = self.nav_to_pose_client.send_goal_async(
    goal_msg, self._feedbackCallback
)
rclpy.spin_until_future_complete(self, send_goal_future)
self.goal_handle = send_goal_future.result()

In [58]:
goal_handle.cancel_goal_async()

In [ ]:
threaded_spinner.stop()

In [70]:
rclpy.spin_until_future_complete(minimal_node, goal_future_pose)

print(goal_future_pose.result())

ClientGoalHandle <id=[ 44  27 101 209  26  83  69 215 185  25 244 153 101 167 169 105], accepted=True, status=2>


In [72]:
# rzeczy z feedbacku możemy odczytywać także jako topic
minimal_node.get_topic_names_and_types()

[('/NavigateToPose/_action/feedback',
  ['nav2_msgs/action/NavigateToPose_FeedbackMessage']),
 ('/NavigateToPose/_action/status', ['action_msgs/msg/GoalStatusArray']),
 ('/amcl/transition_event', ['lifecycle_msgs/msg/TransitionEvent']),
 ('/amcl_pose', ['geometry_msgs/msg/PoseWithCovarianceStamped']),
 ('/assisted_teleop/_action/feedback',
  ['nav2_msgs/action/AssistedTeleop_FeedbackMessage']),
 ('/assisted_teleop/_action/status', ['action_msgs/msg/GoalStatusArray']),
 ('/backup/_action/feedback', ['nav2_msgs/action/BackUp_FeedbackMessage']),
 ('/backup/_action/status', ['action_msgs/msg/GoalStatusArray']),
 ('/behavior_server/transition_event', ['lifecycle_msgs/msg/TransitionEvent']),
 ('/behavior_tree_log', ['nav2_msgs/msg/BehaviorTreeLog']),
 ('/bond', ['bond/msg/Status']),
 ('/bt_navigator/transition_event', ['lifecycle_msgs/msg/TransitionEvent']),
 ('/camera/camera_info', ['sensor_msgs/msg/CameraInfo']),
 ('/camera/image_raw', ['sensor_msgs/msg/Image']),
 ('/camera/image_raw/compr

[Next exercise: 8. Parameters](8.%20Parameters.ipynb)